![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/Spark_NLP_Udemy_MOOC/Healthcare_NLP/GenericSVMClassifierModel.ipynb)

# **GenericSVMClassifierModel**

This notebook will cover the different parameters and usages of `GenericSVMClassifierModel`.

**📖 Learning Objectives:**

1. Background: Understand the 'GenericSVMClassifierModel' annotator.

2. Colab setup.

3. Become comfortable with using the different parameters of the annotator.

**🔗 Helpful Links:**

- Documents : [GenericSVMClassifierModel](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#genericsvmclassifier)

- Python Docs : [GenericSVMClassifierModel](https://nlp.johnsnowlabs.com/licensed/api/python/reference/autosummary/sparknlp_jsl/annotator/classification/generic_svm_classifier/index.html#sparknlp_jsl.annotator.classification.generic_svm_classifier.GenericSVMClassifierModel)

- Scala Docs: [GenericSVMClassifierModel](https://nlp.johnsnowlabs.com/licensed/api/com/johnsnowlabs/nlp/annotators/classification/GenericSVMClassifierModel.html)

- For extended examples of usage, see [Spark NLP Workshop repository](https://colab.research.google.com/drive/12JQuR7cSUeYfCu418REcqkSmkYWq8Efb#scrollTo=Ltt3uq8y9MKR).


## **📜 Background**

`GenericSVMClassifier` is a derivative of [GenericClassifier](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#genericclassifier), which implements [SVM (Support Vector Machine) classification](https://en.wikipedia.org/wiki/Support_vector_machine).

The input to the model is the `FeatureVector` and the output is category annotations with labels and corresponding confidence scores.

The scores are standardized using the logistic function so that they vary between 0 and 1.

## **🎬 Colab Setup**

In [1]:
# Install the johnsnowlabs library to access Spark-NLP for Healthcare
! pip install -q johnsnowlabs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.2/265.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.2/676.2 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.2

In [2]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Please Upload your John Snow Labs License using the button below


Saving 5.3.3.spark_nlp_for_healthcare.json to 5.3.3.spark_nlp_for_healthcare.json


In [3]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

👌 Detected license file /content/5.3.3.spark_nlp_for_healthcare.json
🚨 Outdated Medical Secrets in license file. Version=5.3.3 but should be Version=5.3.2
🚨 Outdated OCR Secrets in license file. Version=5.1.2 but should be Version=5.3.2
👷 Trying to install compatible secrets. Use nlp.settings.enforce_versions=False if you want to install outdated secrets.
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up  John Snow Labs home in /root/.johnsnowlabs, this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
👌 Detected license file /content/5.3.3.spark_nlp_for_healthcare.json
👷 Trying to install compatible secrets. Use nlp.settings.enforce_versions=False if you want to install outdated secrets.
Installing /root/.johnsnowlabs/py_installs/spark_nlp_jsl-5.3.2-py3-none-any.whl to /usr/bin/python3
Installed 1 products:
💊 Spark-Healthcare==5.3.2 installed! ✅ Heal the planet with NLP! 


In [4]:
# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Detected license file /content/5.3.3.spark_nlp_for_healthcare.json
👷 Trying to install compatible secrets. Use nlp.settings.enforce_versions=False if you want to install outdated secrets.
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.3.2, 💊Spark-Healthcare==5.3.2, running on ⚡ PySpark==3.4.0


In [5]:
import pandas as pd

spark

## **🖨️ Input/Output Annotation Types**



- Input: `FEATURE_VECTOR`

- Output: `CATEGORY`

## **🔎 Parameters**

`featureScaling`: Feature scaling method. Possible values are '`zscore`', '`minmax`' or empty (no scaling) (default: )

`multiClass`: Whether to return only the label with the highest confidence score or all labels (default: False, current: False)


`inputCols`: previous annotations columns, if renamed (current: ['features'])

`outputCol`: output annotation column. can be left default. (current: class)



# ✍  Explaining GenericSVMClassifierModel with an Example

## 📑 Sample Data

In [6]:
#downloading sample datasets
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

In [9]:
df_neg= pd.read_csv("ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]

df_pos= pd.read_csv("DRUG-AE.rel", header=None, delimiter="|")
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]

ade_df= pd.concat([df_neg, df_pos])
ade_df.category = ade_df.category.apply(lambda x: True if x=="pos" else False)
ade_df.sample(5, random_state=42)

,text,category
11571,"Nineteen months after diagnosis, her mother g...",False
4577,"An infant girl with choanal atresia, athelia, ...",True
5638,A causal relationship between the administrat...,False
12799,The absence of eosinophils does not exclude t...,False
427,While for ribavirin antidepressant effects are...,True


In [10]:
from pyspark.sql.types import StringType
sample_data = ade_df.sample(n= 100, random_state=1)  # taking 100 rows as sample data
sample_data = sample_data.reset_index(drop=True)
df = spark.createDataFrame(sample_data['text'].tolist(), StringType()).toDF("text")
df.show(5,truncate=60)

+------------------------------------------------------------+
|                                                        text|
+------------------------------------------------------------+
| Multicentric canine lymphoma in a 12-year-old keeshond: ...|
| Pyomyositis is a rare disease, encountered mainly in tro...|
| Both patients subsequently developed markedly elevated E...|
|Bortezomib-induced paralytic ileus is a potential gastroi...|
|However, given the clinically significant result to the i...|
+------------------------------------------------------------+
only showing top 5 rows



In [11]:
sample_data.head()

,text,category
0,Multicentric canine lymphoma in a 12-year-old...,False
1,"Pyomyositis is a rare disease, encountered ma...",False
2,Both patients subsequently developed markedly...,False
3,Bortezomib-induced paralytic ileus is a potent...,True
4,"However, given the clinically significant resu...",True


## **💻Pipeline**

In [12]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer()\
    .setInputCols("document")\
    .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("word_embeddings")

sentence_embeddings = nlp.SentenceEmbeddings() \
    .setInputCols(["document", "word_embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

features_asm = medical.FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

generic_classifier = medical.GenericSVMClassifierModel.pretrained("generic_svm_classifier_ade", "en", "clinical/models")\
    .setInputCols(["features"])\
    .setOutputCol("class")

clf_Pipeline = nlp.Pipeline(stages=[
    document_assembler,
    tokenizer,
    word_embeddings,
    sentence_embeddings,
    features_asm,
    generic_classifier])

data = spark.createDataFrame([["""None of the patients required treatment for the overdose."""],
 ["""I feel a bit drowsy & have a little blurred vision after taking an insulin"""]]).toDF("text")



embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
generic_svm_classifier_ade download started this may take some time.
[OK!]


In [13]:
generic_classifier

GenericSVMClassifierModel_5b8f3a210253

In [14]:
result = clf_Pipeline.fit(data).transform(df)
# result.select("text", "class.result").show(truncate=100)

In [15]:
result.select("text", "class.result").show(truncate=100)

+----------------------------------------------------------------------------------------------------+-------+
|                                                                                                text| result|
+----------------------------------------------------------------------------------------------------+-------+
|                       Multicentric canine lymphoma in a 12-year-old keeshond: chemotherapy options.|[False]|
|                             Pyomyositis is a rare disease, encountered mainly in tropical climates.|[False]|
| Both patients subsequently developed markedly elevated EBV-DNA titers in association with monocl...|[False]|
|Bortezomib-induced paralytic ileus is a potential gastrointestinal side effect of this first-in-c...|[False]|
|However, given the clinically significant result to the interaction between tolazoline and cimeti...| [True]|
|                                              How much do novel antipsychotics benefit the patients?|[False]|
|

In [16]:
preds_df = result.select("text", "class.result").toPandas()
preds_df['result'] = preds_df['result'].apply(lambda x : True if x[0]=='True' else False)
preds_df


,text,result
0,Multicentric canine lymphoma in a 12-year-old...,False
1,"Pyomyositis is a rare disease, encountered ma...",False
2,Both patients subsequently developed markedly...,False
3,Bortezomib-induced paralytic ileus is a potent...,False
4,"However, given the clinically significant resu...",True
...,...,...
95,Subsequently he responded to the West Berlin ...,False
96,All three patients likely experienced electro...,False
97,We report a case of codeine intoxication in th...,False
98,METHODS: The clinical suspicion of chickenpox...,False


In [17]:
from sklearn.metrics import  classification_report
print (classification_report(sample_data['category'], preds_df['result']))

              precision    recall  f1-score   support

       False       0.85      0.93      0.89        72
        True       0.76      0.57      0.65        28

    accuracy                           0.83       100
   macro avg       0.81      0.75      0.77       100
weighted avg       0.82      0.83      0.82       100



### ⏰Running the **GenericSVMClassifierModel** with parameters

In [18]:
print(generic_classifier.explainParams())

classes: Categorization classes (current: ['False', 'True'])
doExceptionHandling: If true, exceptions are handled.
    If exception causing data is passed to the model, a error annotation is emitted which has the exception message.
    Processing continues with the next one.
    This comes with a performance penalty. (default: False)
featureScaling: Feature scaling method. Possible values are 'zscore', 'minmax' or empty (no scaling) (default: )
inputCols: previous annotations columns, if renamed (current: ['features'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
multiClass: Whether to return only the label with the highest confidence score or all labels (default: False, current: False)
outputCol: output annotation column. can be left default. (current: class)
storageRef: storage unique identifier (default: GenericSVMClassifierModel_5b8f3a210253)


### ▶`featureScaling`


buraya feature scaling tanim eklenmesi

In [19]:
generic_classifier = medical.GenericSVMClassifierModel.pretrained("generic_svm_classifier_ade", "en", "clinical/models")\
    .setInputCols(["features"])\
    .setOutputCol("class")\
    .setFeatureScaling('zscore')

clf_Pipeline2 = nlp.Pipeline(stages=[
    document_assembler,
    tokenizer,
    word_embeddings,
    sentence_embeddings,
    features_asm,
    generic_classifier])

generic_svm_classifier_ade download started this may take some time.
[OK!]


In [20]:
result2 = clf_Pipeline2.fit(data).transform(df)
result2.select("text", "class.result","class.metadata").show(truncate=100)   # zscore values

+----------------------------------------------------------------------------------------------------+-------+----------------------------+
|                                                                                                text| result|                    metadata|
+----------------------------------------------------------------------------------------------------+-------+----------------------------+
|                       Multicentric canine lymphoma in a 12-year-old keeshond: chemotherapy options.|[False]|[{confidence -> 0.89175177}]|
|                             Pyomyositis is a rare disease, encountered mainly in tropical climates.|[False]|[{confidence -> 0.97560674}]|
| Both patients subsequently developed markedly elevated EBV-DNA titers in association with monocl...|[False]|[{confidence -> 0.99982864}]|
|Bortezomib-induced paralytic ileus is a potential gastrointestinal side effect of this first-in-c...| [True]| [{confidence -> 0.7110752}]|
|However, given the 

In [21]:
result.select("text", "class.result","class.metadata").show(truncate=100)  # default values

+----------------------------------------------------------------------------------------------------+-------+----------------------------+
|                                                                                                text| result|                    metadata|
+----------------------------------------------------------------------------------------------------+-------+----------------------------+
|                       Multicentric canine lymphoma in a 12-year-old keeshond: chemotherapy options.|[False]|[{confidence -> 0.89175177}]|
|                             Pyomyositis is a rare disease, encountered mainly in tropical climates.|[False]|[{confidence -> 0.97560674}]|
| Both patients subsequently developed markedly elevated EBV-DNA titers in association with monocl...|[False]|[{confidence -> 0.99982864}]|
|Bortezomib-induced paralytic ileus is a potential gastrointestinal side effect of this first-in-c...| [True]| [{confidence -> 0.7110752}]|
|However, given the 

In [22]:
generic_classifier = medical.GenericSVMClassifierModel.pretrained("generic_svm_classifier_ade", "en", "clinical/models")\
    .setInputCols(["features"])\
    .setOutputCol("class")\
    .setFeatureScaling('minmax')

clf_Pipeline3 = nlp.Pipeline(stages=[
    document_assembler,
    tokenizer,
    word_embeddings,
    sentence_embeddings,
    features_asm,
    generic_classifier])

generic_svm_classifier_ade download started this may take some time.
[OK!]


In [23]:
result3 = clf_Pipeline3.fit(data).transform(df)
result3.select("text", "class.result","class.metadata").show(truncate=100)  #minmax values

+----------------------------------------------------------------------------------------------------+-------+----------------------------+
|                                                                                                text| result|                    metadata|
+----------------------------------------------------------------------------------------------------+-------+----------------------------+
|                       Multicentric canine lymphoma in a 12-year-old keeshond: chemotherapy options.| [True]| [{confidence -> 0.6639472}]|
|                             Pyomyositis is a rare disease, encountered mainly in tropical climates.| [True]|[{confidence -> 0.58102447}]|
| Both patients subsequently developed markedly elevated EBV-DNA titers in association with monocl...|[False]| [{confidence -> 0.4665953}]|
|Bortezomib-induced paralytic ileus is a potential gastrointestinal side effect of this first-in-c...| [True]|[{confidence -> 0.71549857}]|
|However, given the 

As shown above, three different `featureScaling` options generated different results and confidence values, and the best one may be chosen according to problem/dataset.

### ▶ `multiClass`

Whether to return only the label with the highest confidence score or all labels

In [24]:
generic_classifier = medical.GenericSVMClassifierModel.pretrained("generic_svm_classifier_ade", "en", "clinical/models")\
    .setInputCols(["features"])\
    .setOutputCol("class")\
    .setMultiClass(True)

clf_Pipeline4 = nlp.Pipeline(stages=[
    document_assembler,
    tokenizer,
    word_embeddings,
    sentence_embeddings,
    features_asm,
    generic_classifier])

generic_svm_classifier_ade download started this may take some time.
[OK!]


In [25]:
result4 = clf_Pipeline4.fit(data).transform(df)
result4.select("text", "class.result","class.metadata").show(truncate=100)

# below, result column is not one value, it is list of values showing confidence of each classes

+----------------------------------------------------------------------------------------------------+-------------+--------------------------------------------------------+
|                                                                                                text|       result|                                                metadata|
+----------------------------------------------------------------------------------------------------+-------------+--------------------------------------------------------+
|                       Multicentric canine lymphoma in a 12-year-old keeshond: chemotherapy options.|[False, True]|  [{confidence -> 0.3259712}, {confidence -> 0.6639472}]|
|                             Pyomyositis is a rare disease, encountered mainly in tropical climates.|[False, True]|[{confidence -> 0.33836725}, {confidence -> 0.58102447}]|
| Both patients subsequently developed markedly elevated EBV-DNA titers in association with monocl...|[False, True]| [{confidence 